In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service as EdgeService  # Import EdgeService
from bs4 import BeautifulSoup
import pandas as pd  # for storing data in a structured way (optional)

def extract_station_data_from_page(driver):
    """Extracts station data from the current page of the table."""
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', class_='stn-dir-list-tbl')  # Find the table

    if table is None:
        print("Table not found on this page.")
        return []

    station_data = []
    rows = table.find_all('tr')  # Get all table rows

    # Extract header information
    header_row = rows[0]
    header_cells = header_row.find_all('th')
    headers = [cell.text.strip() for cell in header_cells]

    for row in rows[1:]:  # Skip the header row
        cells = row.find_all('td')
        if not cells:
            continue  # Skip empty rows

        row_data = {}
        for i, cell in enumerate(cells):
            if i == 1:  # Handle the station name (which contains a link)
                link = cell.find('a')
                if link:
                    row_data[headers[i]] = link.text.strip()  # Extract link text
                else:
                    row_data[headers[i]] = cell.text.strip()
            else:
                row_data[headers[i]] = cell.text.strip()

        station_data.append(row_data)

    return station_data

def extract_all_pages_data(driver, letter):
    """Extracts station data from all pages for a given letter."""
    all_data = []
    page_num = 1  # Start at page 1

    while True:
        print(f"Extracting data from page {page_num} for letter '{letter}'")
        page_data = extract_station_data_from_page(driver)  #Extract data from current page
        if not page_data:
            print(f"No data found on page {page_num} for letter '{letter}'. Stopping.")
            break

        all_data.extend(page_data)

        # Find the "Next" button and click it. Use a try-except block in case the button is disabled or not found
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next →")
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled, no more pages")
                break # Exit if no next page
            next_button.click()
            driver.implicitly_wait(5) #wait for next page to load
            page_num += 1
        except Exception as e:
            print(f"Next button not found or could not be clicked. Assuming last page reached. {e}")
            break

    return all_data


# def extract_station_data_for_letter(url, letter):
#     """Extracts all station data for a given letter, handling multiple pages."""

#     driver = None  # Initialize driver outside the try block
#     try:
#         # 1. Set up Selenium Edge WebDriver
#         driver = webdriver.Edge()
#         driver.get(url)

#         # 2. Click on the specified letter
#         letter_element = driver.find_element(By.XPATH, f'//a[@class="StationName" and @data-station-name="{letter}"]')
#         letter_element.click()
#         driver.implicitly_wait(5)

#         # 3. Extract data from all pages
#         all_data = extract_all_pages_data(driver, letter)  # Modified function call
#         return all_data

#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return []

#     finally:
#         # 5. Close the WebDriver
#         if driver:  # Check if the driver was initialized
#             driver.quit()
#             # Modify the URL to include the letter and page number directly
def extract_station_data_for_letter(url, letter):
    """Extracts all station data for a given letter, handling multiple pages."""

    driver = None  # Initialize driver outside the try block
    try:
        # 1. Set up Selenium Edge WebDriver
        driver = webdriver.Edge()

        # 2. Extract data from all pages
        all_data = []
        page_num = 1
        while True:
            # Construct the URL with the letter and page number
            page_url = f"{url}?name={letter}&page={page_num}"
            print(f"Accessing URL: {page_url}")
            driver.get(page_url)
            driver.implicitly_wait(5)

            # Extract data from the current page
            page_data = extract_station_data_from_page(driver)
            if not page_data:
                print(f"No data found on page {page_num} for letter '{letter}'. Stopping.")
                break

            all_data.extend(page_data)
            page_num += 1

        return all_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    finally:
        # 3. Close the WebDriver
        if driver:  # Check if the driver was initialized
            driver.quit()
# Example Usage:
url = "https://www.railyatri.in/stations"  # Replace with the actual URL of the webpage
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"  # All letters to process

all_data = {} # Store data for all letters

for letter in letters:
    
    
    data = extract_station_data_for_letter(url, letter)

    if data:
        print(f"Found {len(data)} stations starting with {letter}")
        all_data[letter] = data # Store data for the letter
    else:
        print(f"No data found for letter '{letter}'.")
        all_data[letter] = []

# Optional: Convert all data to Pandas DataFrame and save to CSV
all_dfs = {}
for letter, data in all_data.items():
    all_dfs[letter] = pd.DataFrame(data)

# Combine all dataframes into a single dataframe:
combined_df = pd.concat(all_dfs.values(), ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv("all_stations.csv", index=False)
print("Data saved to all_stations.csv")



In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# --- Configuration ---

PAGE_URL = "https://www.ixigo.com/hotels"
DESTINATION_CITY = "Lucknow"

# --- WebDriver Setup ---

driver = webdriver.Edge()
driver.maximize_window()


try:
    print(f"Navigating to {PAGE_URL}...")
    driver.get(PAGE_URL)

    wait = WebDriverWait(driver, 10)

    # --- 1. Handle Destination Input ---
    print(f"Locating and updating Destination input...")

    # Retry logic for StaleElementReferenceException
    for attempt in range(3):  # Try up to 3 times
        try:
            destination_input_locator = (By.XPATH, "//p[text()='Destination']/following-sibling::input")
            destination_input = wait.until(EC.element_to_be_clickable(destination_input_locator))

            # *CLICK THE INPUT FIELD FIRST*
            destination_input.click()
            print("Clicked on the Destination input field.")

            destination_input.clear()
            destination_input.send_keys(DESTINATION_CITY)
            print(f"Entered '{DESTINATION_CITY}' into Destination input.")
            break  # If successful, exit the loop
        except StaleElementReferenceException:
            print(f"StaleElementReferenceException occurred (attempt {attempt + 1}). Retrying...")
            time.sleep(1)  # Wait a bit before retrying
    else:  # If the loop completes without a 'break'
        print("Failed to locate and update Destination input after multiple attempts.")
        raise


    try:

        # Use the provided HTML to locate the correct element:
        mumbai_option_locator = (By.XPATH, f"//div[@data-testid='{DESTINATION_CITY}' and @role='button']")
        mumbai_option = wait.until(EC.element_to_be_clickable(mumbai_option_locator))
        mumbai_option.click()
    except TimeoutException:
        print("Error: Timed out waiting for the 'Mumbai' dropdown item to be clickable.")
        raise
    except Exception as e:
        print(f"Error occurred while selecting 'Mumbai' from dropdown: {e}")
        raise

    # --- 3. Handle Search Button ---
    print("Locating Search button...")
    search_button_locator = (By.CSS_SELECTOR, 'button[data-testid="search-hotels"]')
    search_button = wait.until(EC.element_to_be_clickable(search_button_locator))
    print("Clicking Search button...")
    search_button.click()
    print("Search initiated successfully!")

    time.sleep(5)
    
    new_url = driver.current_url
    print(f"New URL: {new_url}")

except TimeoutException:
    print("Error: Timed out waiting for one or more elements to load or become interactive.")

except Exception as e:
    print(f"An error occurred: {e}")

## Flights

In [ ]:
import json
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from urllib.parse import quote
from bs4 import BeautifulSoup  # Import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import deque



edge_driver_path = r"D:\msedgedriver.exe"  # Replace with your path
edge_options = EdgeOptions()
#edge_options.add_argument("--headless=new")  # Uncomment for headless mode
edge_options.add_argument("--start-maximized")
edge_options.add_argument("--force-device-scale-factor=0.05")
edge_service = EdgeService(executable_path=edge_driver_path)
driver = webdriver.Edge(service=edge_service, options=edge_options)


def access_ixigo(from_city="LKO", to_city=" ,
                 date="17042025", return_date="22042025",
                 adults=2, children=0, infants=0, flight_class="e",
                 hbs=True):
    """
    Accesses the Ixigo flight search website and extracts flight information
    *including* structured data extracted from the HTML of each flight card.

    Args:
        from_city (str): Departure city code.
        to_city (str): Destination city code.
        date (str): Departure date (DDMMYYYY).
        return_date (str): Return date (DDMMYYYY).
        adults (int): Number of adults.
        children (int): Number of children.
        infants (int): Number of infants.
        flight_class (str): Flight class (e.g., "e" for economy).
        hbs (bool): Whether to search for hotel-bundled savings.
    """

    # Construct the URL (using f-strings for readability)
    base_url = "https://www.ixigo.com/search/result/flight"
    params = {
        "from": from_city,
        "to": to_city,
        "date": date,
        "returnDate": return_date,
        "adults": adults,
        "children": children,
        "infants": infants,
        "class": flight_class,
        "source": "Search+Form",  # Note: The space is encoded as '+'
        "hbs": str(hbs).lower()  # Convert boolean to lowercase string
    }

    # Build the query string
    query_string = "&".join([f"{key}={quote(str(value))}" for key, value in params.items()])  # use quote to encode spaces and special characters
    url = f"{base_url}?{query_string}"

    try:
        driver.get(url)
        time.sleep(5)  # Allow time for page to load and flights to populate
        

        print("Page Title:", driver.title)

        flight_data = extract_ixigo_flight_data(driver)

        # with open("ixigo_flights.json", "w", encoding="utf-8") as f:
        #     json.dump(flight_data, f, indent=4, ensure_ascii=False)

        print("Flight data saved to ixigo_flights.json")

    except Exception as e:
        print(f"An error occurred: {e}")
    # finally:
        #driver.quit()  # Ensure the driver quits, especially after an error


def extract_ixigo_flight_data(driver):
    """
    Extracts flight data from the Ixigo search results page,
    extracting both structured data and using BeautifulSoup to parse the HTML.

    Args:
        driver: Selenium WebDriver instance.

    Returns:
        A list of dictionaries, where each dictionary represents a flight.
    """
    # WebDriverWait(driver, 10).until(
    #         EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="shadow-[0px_2px_5px_0px_rgba(0,0,0,0.10)] p-15 mb-20  rounded-10 relative cursor-pointer bg-white border border-white transition-all duration-300 ease-in hover:scale-[1.01] hover:shadow-300 hover:duration-300 hover:ease-out"]'))
    #     )

    flight_data = []  # Use deque for efficient appending
    flight_cards_data = []
    flight_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'cursor-pointer')]")
    # for  flight_card in flight_cards:
    #     print(flight_card)
    print(len(flight_cards))
    for card in flight_cards:
        try:
            # **Scroll to the element before extracting**
            # driver.execute_script("arguments[0].scrollIntoView();", card)
            time.sleep(0.5)  # Give it a brief pause to fully load after scrolling

            # **Extract the HTML**
            html_string = card.get_attribute('outerHTML')

            # **Parse the HTML with BeautifulSoup**
            flight = extract_flight_data_from_html(html_string)  # Use the parsing function

            flight_data.append(flight)

        except Exception as e:
            print(f"Error extracting data from a flight card: {e}")
            continue

    return flight_data


def extract_flight_data_from_html(html_string):
    """
    Extracts specific flight data from the HTML of a flight card using BeautifulSoup.
    This function extracts:
    - Fastest flight indicator
    - Airline
    - Departure Time
    - Arrival Time
    - Flight Duration
    - Stops
    - Price
    """
    soup = BeautifulSoup(html_string, 'html.parser')

    flight = {}

    # Fastest flight indicator
    fastest_element = soup.find('div', class_='text-selection-outline border-selection-outline min-h-20 icon-sm body-xs inline-flex items-center font-normal rounded-full px-px border border-solid bg-white')
    flight['fastest'] = True if fastest_element else False

    # Airline
    airline_element = soup.find('p', class_='body-sm body-sm')
    flight['airline'] = airline_element.text.strip() if airline_element else None

    # Departure Time
    departure_time_element = soup.find('h5', class_='h5 text-primary font-medium')
    flight['departure_time'] = departure_time_element.text.strip() if departure_time_element else None

    # Arrival Time (look for the element with GOX as destination which is now generic GOI)
    arrival_time_element = soup.find_all('h5', class_='h5 text-primary font-medium')[1]
    flight['arrival_time'] = arrival_time_element.text.strip() if len(soup.find_all('h5', class_='h5 text-primary font-medium')) > 1 and arrival_time_element else None #get the second element

    # Flight Duration
    duration_element = soup.find('p', class_='body-sm text-secondary')
    flight['duration'] = duration_element.text.strip() if duration_element else None

    # Stops
    stops_element = soup.find_all('p', class_='body-sm text-secondary')
    flight['stops'] = stops_element[1].text.strip() if len(stops_element) > 1 and stops_element[1] else None #get the second element


    # Price
    price_element = soup.find('h5', {'data-testid': 'pricing'})
    flight['price'] = price_element.text.strip() if price_element else None

    return flight

# Example usage:
if __name__ == "__main__":
    access_ixigo()